In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

/tmp/ipykernel_6901/3914160640.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
!pwd

/root/3A/ProjetLong/OrthopteraStridulationClassifier/Notebooks/XgBoost


In [5]:
## Pour train : les données de 1 min qui ont été streched
path_train_audio_files = "../../../DataSet/NewDataSet5sec/Selection soundscapes 1 min/"
path_train_csv = "../../../DataSet/NewYolo/train_csv.csv" 

## Pour test : les boundingbox de Yolo
path_test_csv = "../../../DataSet/NewYolo/1min_to_5s_yolo.csv"

In [6]:
df_train = pd.read_csv(path_train_csv) 
df_test = pd.read_csv(path_test_csv) 

In [7]:
## get_sampling_rate of the recordings and the code_unique 
sampling_dict = dict()
for _, row in df_train.iterrows() : 
    code = row['code_unique']
    if code not in sampling_dict : 
        sampling_dict[code] = row['sampling_rate']

print(sampling_dict)

{'Orthoptera_cdpotsa09FR_05X2022_806': 38400, 'Orthoptera_plmarsa31FR_12V2023_1213': 44100, 'Orthoptera_cdpotsa09FR_05X2022_830': 38400, 'Orthoptera_guneust09FR_10VII2023_1329': 96000, 'Orthoptera_guneust09FR_10VII2023_1332': 96000, 'Orthoptera_adkarsa31FR_21IX2022_1023': 38400, 'Orthoptera_hotrrsa31FR_21IX2022_706': 38400, 'Orthoptera_hotrrsa31FR_21IX2022_991': 38400, 'Orthoptera_cdchcau09FR_18IX2022_1055': 96000, 'Orthoptera_cdchcau09FR_18IX2022_1048': 96000, 'Orthoptera_cdchcau09FR_18IX2022_1052': 96000, 'Orthoptera_XXXXigeXXES_01VII2022_331': 44100, 'Orthoptera_XXXXigeXXES_01VII2022_332': 44100, 'Orthoptera_XXXXigeXXES_01VIII2022_900': 44100, 'Orthoptera_XXXXigeXXES_02VIII2022_901': 44100, 'Orthoptera_XXXXjacXXES_26VIII2022_1035': 96000, 'Orthoptera_XXXXporXXES_06V2022_123': 44100, 'Orthoptera_XXXXporXXES_06V2022_124': 44100, 'Orthoptera_XXXXporXXES_06V2022_125': 44100, 'Orthoptera_XXXXporXXES_30IV2022_117': 44100, 'Orthoptera_XXXXporXXES_30IV2022_118': 44100, 'Orthoptera_XXXXbid07

In [8]:
######## Preparer les données de Yolo pour retrouver les fréquences à partir des pixels
def prepare_yolo_data(df_yolo) :     
    colonnes = ["unique_code", "max_frequency", "min_frequency","img_width", "img_height", "x1", "x2", "y1", "y2", "duree", "start_split", "end_split", "predicted_label"]
    new_df = pd.DataFrame(columns=colonnes)
    new_df["unique_code"] = df_yolo["unique_code"]
    new_df["start_split"] = df_yolo["start_split"]
    new_df["end_split"] = df_yolo["end_split"]
    new_df["x1"] = df_yolo["x1"]
    new_df["x2"] = df_yolo["x2"]
    new_df["y1"] = df_yolo["y1"]
    new_df["y2"] = df_yolo["y2"]
    new_df["img_width"] = df_yolo["img_width"]
    new_df["img_height"] = df_yolo["img_height"]

    based_sr = 48000
    for ind, row in new_df.iterrows():
        width = row["img_width"]
        height = row["img_height"]
        x1 = row["x1"]
        x2 = row["x2"]
        y1 = row["y1"]
        y2 = row["y2"]
        start = row["start_split"]
        end = row["end_split"]
        code = row["unique_code"]
        ratio = min(sampling_dict[code]/based_sr,1)
        minfreq = ((height - y2)*based_sr*ratio) / (height * 2)
        maxfreq = ((height - y1)*based_sr*ratio) / (height * 2)

        new_df.loc[ind, "min_frequency"] = minfreq 
        new_df.loc[ind, "max_frequency"] = maxfreq 
        new_df.loc[ind, "duree"] = end - start
    return new_df


In [9]:
new_df = prepare_yolo_data(df_test)
print(new_df)

                              unique_code max_frequency min_frequency  \
0        Anura_estalep09FR_27III2022_1330   2177.560976    351.219512   
1        Anura_estalep09FR_27III2022_1345    1240.97561     23.414634   
2        Anura_estalep09FR_27III2022_1345   1685.853659     23.414634   
3        Anura_estalep09FR_27III2022_1345   1170.731707           0.0   
4        Anura_estalep09FR_27III2022_1345   1030.243902           0.0   
...                                   ...           ...           ...   
2295  Orthoptera_XXXXsma07FR_27IV2022_112  18070.243902   3183.804878   
2296  Orthoptera_XXXXsma07FR_27IV2022_112  18156.292683   3248.341463   
2297  Orthoptera_XXXXsma07FR_27IV2022_112  18048.731707   3205.317073   
2298  Orthoptera_XXXXsma07FR_27IV2022_112  18070.243902   3183.804878   
2299  Orthoptera_XXXXsma07FR_27IV2022_112  18048.731707   3226.829268   

      img_width  img_height  x1   x2   y1    y2 duree  start_split  end_split  \
0           469        1025  45   95  932 

In [42]:
print(new_df['min_frequency'].min())
print(new_df['min_frequency'].max())
print(new_df['max_frequency'].min())
print(new_df['max_frequency'].max())

0.0
17373.658536585364
608.780487804878
24000.0


In [43]:
path_1min = "../../../DataSet/NewYolo/train_stretch.csv"

df_train = pd.read_csv(path_1min)
#df_train = df_train[df_train["label_class"] == "Insecta"]

label_encoder = LabelEncoder()
label_encoder.fit(df_train['label'])

y_label = label_encoder.transform(df_train['label'])

df_train = df_train[['max_frequency', 'min_frequency']]
df_train['min_frequency'] = df_train['min_frequency'].astype(int)
df_train['max_frequency'] = df_train['max_frequency'].astype(int)

# Gestion des valeurs manquantes
#df.fillna(df.mean(), inplace=True)
df_train = df_train.dropna()

# Gestion des variables catégorielles
df_train = pd.get_dummies(df_train)

X_train = df_train

#################### Préparer X_test avec les données ci-dessus 

new_dftest = new_df[['max_frequency', 'min_frequency']]
new_dftest['min_frequency'] = new_dftest['min_frequency'].astype(int)
new_dftest['max_frequency'] = new_dftest['max_frequency'].astype(int)

# Gestion des valeurs manquantes
#new_dftest.fillna(new_dftest.mean(), inplace=True)
new_dftest = new_dftest.dropna()

# Gestion des variables catégorielles
new_dftest = pd.get_dummies(new_dftest)

X_test = new_dftest

/tmp/ipykernel_11871/2747467818.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dftest['min_frequency'] = new_dftest['min_frequency'].astype(int)
/tmp/ipykernel_11871/2747467818.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_dftest['max_frequency'] = new_dftest['max_frequency'].astype(int)


In [44]:
print(X_test)

      max_frequency  min_frequency
0              2177            351
1              1240             23
2              1685             23
3              1170              0
4              1030              0
...             ...            ...
2295          18070           3183
2296          18156           3248
2297          18048           3205
2298          18070           3183
2299          18048           3226

[2300 rows x 2 columns]


In [45]:
# XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=2)

xgb_model.fit(X_train, y_label)

predictions_xgb = xgb_model.predict(X_test)

predictions_label_xgb = label_encoder.inverse_transform(predictions_xgb)

In [46]:
print(predictions_label_xgb)

['Mouche' 'Mouche' 'Mouche' ... 'Aves sp.' 'Aves sp.' 'Sylvia atricapilla']


In [47]:
# Random forest

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")

rf_classifier.fit(X_train, y_label)

predictions_randomf = rf_classifier.predict(X_test)

predictions_label_randomf = label_encoder.inverse_transform(predictions_randomf)


In [49]:
df = pd.read_csv(path_train_csv) 
df_randomf = new_df
df_randomf['predicted_label'] = predictions_label_randomf

df_xgb= new_df
df_xgb['predicted_label'] = predictions_label_xgb
print(df_xgb)

                              unique_code max_frequency min_frequency  \
0        Anura_estalep09FR_27III2022_1330   2177.560976    351.219512   
1        Anura_estalep09FR_27III2022_1345    1240.97561     23.414634   
2        Anura_estalep09FR_27III2022_1345   1685.853659     23.414634   
3        Anura_estalep09FR_27III2022_1345   1170.731707           0.0   
4        Anura_estalep09FR_27III2022_1345   1030.243902           0.0   
...                                   ...           ...           ...   
2295  Orthoptera_XXXXsma07FR_27IV2022_112  18070.243902   3183.804878   
2296  Orthoptera_XXXXsma07FR_27IV2022_112  18156.292683   3248.341463   
2297  Orthoptera_XXXXsma07FR_27IV2022_112  18048.731707   3205.317073   
2298  Orthoptera_XXXXsma07FR_27IV2022_112  18070.243902   3183.804878   
2299  Orthoptera_XXXXsma07FR_27IV2022_112  18048.731707   3226.829268   

      img_width  img_height  x1   x2   y1    y2 duree  start_split  end_split  \
0           469        1025  45   95  932 

In [50]:
# Calcul d'une métrique : 
# Dans un soundcsape, on considère ici que le label est bien prédit s'il est effectivement présent dans la minute d'enregistrement.
# A affiner pour vérifier bounding box par bounding box
def acc(sub_df) : 
    val = 0 
    for _,row in sub_df.iterrows() : 
        pred = row['predicted_label']
        code = row['unique_code']
        inter_df = df[df['code_unique'] == code]
        for _, row2 in inter_df.iterrows() : 
            truth = row2['label'] 
            if truth == pred : 
                val +=1 
                break
    result = val/len(sub_df['predicted_label'])
    return result 

acc_xgb  = acc(df_xgb)
print(acc_xgb)
acc_randf = acc(df_randomf)
print(acc_randf)

0.3565217391304348
0.3565217391304348
